<a href="https://colab.research.google.com/github/hfelizzola/Timetabling-Unisalle/blob/main/Time_Tabling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Tabling Model

In [1]:
# Instalar paquete ortools
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 4.9 MB/s 


In [17]:
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [16]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [18]:
os.chdir('/gdrive/MyDrive/Colab Notebooks/Time Tabling')

In [19]:
!ls

Time_Tabling.ipynb


In [11]:
# Crear el modelo
solver = pywraplp.Solver.CreateSolver('SCIP')

## Conjuntos

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky">   <br>Subíndice   </th>
    <th class="tg-0pky">   <br>Descripción   </th>
    <th class="tg-0pky">   <br>Dominio   </th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">   <br>i   </td>
    <td class="tg-0pky">   <br>Franjas Horarias   </td>
    <td class="tg-0pky">   <br>i = 1,2,…,11   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>j   </td>
    <td class="tg-0pky">   <br>Asignatura   </td>
    <td class="tg-0pky">   <br>j = 1,2,…,90   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>k   </td>
    <td class="tg-0pky">   <br>Docentes   </td>
    <td class="tg-0pky">   <br>k = 1,2,…,29   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>n   </td>
    <td class="tg-0pky">   <br>Días hábiles   </td>
    <td class="tg-0pky">   <br>n = 1,2,…,6   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>m   </td>
    <td class="tg-0pky">   <br>Semestre al que   pertenecer una asignatura   </td>
    <td class="tg-0pky">   <br>m = 1,2,…,10   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>t   </td>
    <td class="tg-0pky">   <br>Bloques de franjas   horarias de dos horas   </td>
    <td class="tg-0pky">   <br>t = 1,2,…,5   </td>
  </tr>
  <tr>
    <td class="tg-0pky">   <br>q   </td>
    <td class="tg-0pky">   <br>Bloques de franjas   horarias de tres horas   </td>
    <td class="tg-0pky">   <br>q = 1,2,3   </td>
  </tr>
</tbody>
</table>

In [147]:
franja_horaria = ['F'+str(i) for i in np.arange(1,12,1)] # i
asignatura = ['M'+str(i) for i in np.arange(1,91,1)] # j
docente = ['P'+str(i) for i in np.arange(1,30,1)] # k
dia = ['D'+str(i) for i in np.arange(1,7,1)] # n
semestre_asignatura = ['S'+str(i) for i in np.arange(1,12,1)] # m
bloques_dos_horas = ['T'+str(i) for i in np.arange(1,6,1)] # t
bloques_tres_horas = ['Q'+str(i) for i in np.arange(1,4,1)] # Q

## Parametros

|      Parámetro     |      Descripción                                                                                                                                 |      subíndice     |
|--------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|--------------------|
|     INTH           |     Número de franjas horarias que se deben dictar de forma   presencial, semanalmente, de la asignatura j.                                      |     j              |
|     PMAX           |     Número de franjas horarias que debe dictar como máximo, el   docente k, a la semana                                                          |     k              |
|     PMIN           |     Número de franjas horarias que debe dictar como mínimo, el   docente k, a la semana                                                          |     k              |
|     DOSF           |     Parámetro binario donde 1 significa que la asignatura j se dicta en bloques de franjas   horarias de dos horas y 0 de lo contrario           |     j              |
|     TRESF          |     Parámetro binario donde 1 significa que la asignatura j se dicta en bloques de franjas   horarias de tres horas y donde 0 de lo contrario    |     j              |
|     MS             |     Matriz binaria donde 1 significa que la asignatura j pertenece al semestre m y 0 de lo contrario                                             |     j,m            |
|     MP             |     Matriz binaria donde 1 significa que la asignatura j puede ser asignada al docente k y donde 0 de lo contrario                               |     j,k            |
|     FP             |     Matriz binaria donde 1 significa que en el día n en la franja i está disponible el docente k   y 0 de lo contrario                           |     n,i,k          |
|     CFM            |     Matriz con ponderación de costo estimado de insatisfacción   de asignar en el día n en la   franja i la asignatura j                         |     n,i,j          |
|     MSF            |     Máximo de salones disponibles por franja horaria para   asignar las diferentes asignaturas                                                   |                    |
|     F              |     Costo total estimado de insatisfacción generado por el   horario asignado                                                                    |                    |

In [148]:
INTH = pd.read_excel('input_data.xlsx', sheet_name='INTH')
INTH

,asignatura,INTH
0,M1,2.0
1,M2,4.0
2,M3,4.0
3,M4,6.0
4,M5,4.0
...,...,...
85,M86,3.0
86,M87,3.0
87,M88,3.0
88,M89,3.0


In [149]:
PMAX = pd.read_excel('input_data.xlsx', sheet_name='PMAX')
PMAX

,docente,PMAX
0,P1,20.0
1,P2,20.0
2,P3,20.0
3,P4,20.0
4,P5,20.0
5,P6,20.0
6,P7,20.0
7,P8,20.0
8,P9,18.0
9,P10,18.0


In [150]:
PMIN = pd.read_excel('input_data.xlsx', sheet_name='PMIN')
PMIN

,docente,PMIN
0,P1,0.0
1,P2,0.0
2,P3,0.0
3,P4,0.0
4,P5,0.0
5,P6,0.0
6,P7,0.0
7,P8,0.0
8,P9,0.0
9,P10,0.0


In [151]:
DOSF = pd.read_excel('input_data.xlsx', sheet_name='DOSF')
DOSF

,asignatura,DOSF
0,M1,1.0
1,M2,1.0
2,M3,1.0
3,M4,1.0
4,M5,1.0
...,...,...
85,M86,0.0
86,M87,0.0
87,M88,0.0
88,M89,0.0


In [152]:
TRESF = pd.read_excel('input_data.xlsx', sheet_name='TRESF')
TRESF

,asignatura,TRESF
0,M1,0.0
1,M2,0.0
2,M3,0.0
3,M4,0.0
4,M5,0.0
...,...,...
85,M86,1.0
86,M87,1.0
87,M88,1.0
88,M89,1.0


In [153]:
MS = pd.read_excel('input_data.xlsx', sheet_name='MS')
MS

,asignatura,semestre_asignatura,MS
0,M1,S1,0.0
1,M2,S1,0.0
2,M3,S1,0.0
3,M4,S1,0.0
4,M5,S1,0.0
...,...,...,...
985,M86,S11,0.0
986,M87,S11,0.0
987,M88,S11,0.0
988,M89,S11,0.0


In [154]:
MP = pd.read_excel('input_data.xlsx', sheet_name='MP')
#MP = MP.melt(id_vars='asignatura', value_vars=docente, var_name='docente', value_name='MP')
MP

,asignatura,docente,MP
0,M1,P1,1.0
1,M2,P1,0.0
2,M3,P1,0.0
3,M4,P1,0.0
4,M5,P1,0.0
...,...,...,...
2605,M86,P29,0.0
2606,M87,P29,0.0
2607,M88,P29,0.0
2608,M89,P29,0.0


In [155]:
FP = pd.read_excel('input_data.xlsx', sheet_name='FP')
#FP.rename(columns={'Unnamed: 0':'dia','Unnamed: 1':'franja_horaria'}, inplace=True)
#FP = FP.melt(id_vars=['dia','franja_horaria'], value_vars=docente, var_name='docente', value_name='FP')
FP

,dia,franja_horaria,docente,FP
0,D1,F1,P1,1.0
1,D1,F2,P1,1.0
2,D1,F3,P1,1.0
3,D1,F4,P1,1.0
4,D1,F5,P1,1.0
...,...,...,...,...
865,D6,F1,P29,1.0
866,D6,F2,P29,1.0
867,D6,F3,P29,1.0
868,D6,F4,P29,0.0


In [156]:
# CFM	Matriz con ponderación de costo estimado de insatisfacción de asignar en el día n en la franja i la asignatura j
CFM = pd.read_excel('input_data.xlsx', sheet_name='CFM')
#CFM.fillna(10, inplace=True)
#CFM = CFM.melt(id_vars=['dia','franja_horaria'], value_vars=asignatura, var_name='asignatura', value_name='CFM')
CFM

,dia,franja_horaria,asignatura,CFM
0,D1,F1,M1,8.0
1,D1,F2,M1,5.0
2,D1,F3,M1,6.0
3,D1,F4,M1,4.0
4,D1,F5,M1,5.0
...,...,...,...,...
2695,D6,F1,M90,8.0
2696,D6,F2,M90,3.0
2697,D6,F3,M90,4.0
2698,D6,F4,M90,10.0


In [157]:
# Número de franjas horarias de la asignatura j
#TRESF = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
#temp = pd.DataFrame({'asignatura':asignatura, 'TRESF':TRESF})
CFM.to_excel('consulta.xlsx', index=False)

In [158]:
90*29

2610

## Variables de Decisión

Se utiliza un total de cuatro variables binarias. 
- La primera consiste en las variables básicas $X_{(i,j,k,n)}$, la cual toma el valor de 1 si en la franja horaria i se asigna la asignatura j al docente k en el día n. 
- El segundo conjunto comprende las variables auxiliares, la variable $Y_{(j,k)}$, la cual toma el valor de 1 si el docente j va a dictar la asignatura k. 
- La variable $H_{(j,k,n,t)}$, H toma el valor de 1 si se activa para asignatura j dictada por el docente k en el día n. 
- La variable $G_{(j,k,n,q)}$, toma el valor de 1 si se activa para asignatura j dictada por el docente k en el día n. 

Las últimas dos variables se hacen necesarias ya que la programación de asignaturas de 4 horas se hace en 2 bloques de 2 horas mientras, las asignaturas de 3 horas se realizan en un solo bloque.

In [159]:
# Creación de la variables X(i,j,k,n)
X = {}
for i in franja_horaria:
  for j in asignatura:
    for k in docente:
      for n in dia:
        X[i, j, k, n] = solver.IntVar(0, 1, '')

In [160]:
# Creación de la variables Y(j,k)
Y = {}
for j in asignatura:
  for k in docente:
      Y[j, k] = solver.IntVar(0, 1, '')

In [161]:
# Creación de la variables H(j,k,n,t)
H = {}
for j in asignatura:
  for k in docente:
    for n in dia:
      for t in bloques_dos_horas:
        H[j, k, n, t] = solver.IntVar(0, 1, '')

In [162]:
# Creación de la variables H(j,k,n,t)
# Creación de la variables H(j,k,n,t)
G = {}
for j in asignatura:
  for k in docente:
    for n in dia:
      for q in bloques_tres_horas:
        G[j, k, n, q] = solver.IntVar(0, 1, '')